<a href="https://colab.research.google.com/github/Zilch123/Aruco-3D-position-Validation/blob/main/ArucoValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Author1: Timoth Dev 
# Author2: Arjun Ram 
# GNU 3.0 Licence 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install opencv-contrib-python
# !git clone https://github.com/ddelago/Aruco-Marker-Calibration-and-Pose-Estimation.git

In [3]:
import os
import sys
import pickle
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2.aruco as aruco
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy import stats

plt.rcParams["figure.figsize"] = (10,5)


Type in your drive path to Data, path is the same till MyDrive

In [4]:
cd drive/MyDrive/Aruco\ CMC\ project/Data/

/content/drive/.shortcut-targets-by-id/1bOHe3Wct63zxzPGjAqK-HttE3BVPIieY/Data


In [5]:
ls Subject\ 1/Subject\ 1/Sub\ 1\ SB

color.pickle  subject1.avi              subject1_color.avi  subject1_depth.avi
prm.pickle    subject1_color_aruco.avi  subject1_color.mp4  xyz.pickle


In [60]:
def read_calib(RotMatrix):
    return np.matrix(RotMatrix[0]), np.matrix(RotMatrix[1])


def kinect2table(orig,R_T,kinect_joint):
    Local_co = [np.matmul(R_T.T,(x-orig).T) for x in kinect_joint]
    return(np.array(Local_co).squeeze())


def my_filter(x):
    return savgol_filter(x, 3, 1)


In [64]:
with open("rotMat_new.pickle",'rb') as f:
    RotMatrix = []
    while True:
        try:
            RotMatrix.append(pickle.load(f))
            
        except EOFError:
            print(RotMatrix[1])
            RotMatrix[1][[0, 2]] = -RotMatrix[1][[2, 0]]
            RotMatrix[1][1] = - RotMatrix[1][1]
            break

[[ 0.99871538 -0.02327689 -0.023868  ]
 [-0.00993747 -0.95835113 -0.27631295]
 [ 0.04968733  0.27619518 -0.96077129]]


In [65]:
RotMatrix[1]

array([[-0.04968733, -0.27619518,  0.96077129],
       [ 0.00993747,  0.95835113,  0.27631295],
       [-0.99871538,  0.02327689,  0.023868  ]])

In [66]:
orig, R_T = read_calib(RotMatrix)

In [67]:
print(orig,'\n\n', R_T)

[[-0.141 -0.251  1.404]] 

 [[-0.04968733 -0.27619518  0.96077129]
 [ 0.00993747  0.95835113  0.27631295]
 [-0.99871538  0.02327689  0.023868  ]]


In [68]:
# https://github.com/ddelago/Aruco-Marker-Calibration-and-Pose-Estimation
# Load the Github camera calib profile 
f = open("Aruco-Marker-Calibration-and-Pose-Estimation/calibration/CameraCalibration.pckl", 'rb')
(cameraMatrix, distCoeffs, _, _) = pickle.load(f, encoding='latin1')
f.close()

# Kinect Calib data
# https://github.com/shanilfernando/VRInteraction/blob/master/calibration/rgb_calibration.yaml
# cameraMatrix = np.array([[ 1.0327407219495085e+03, 0., 9.5685930301206076e+02],
#                          [ 0., 1.0323427647512485e+03, 5.3914133979587950e+02],
#                          [0., 0., 1. ]])
# distCoeffs=np.array([[ 2.0429028189430772e-02, -7.8399448855923665e-03, 
#                      -3.0949667668667908e-03, 2.1848647826377197e-03, -5.2344198896187882e-02 ]])

print(cameraMatrix, distCoeffs )


[[1.44095165e+03 0.00000000e+00 9.40438869e+02]
 [0.00000000e+00 1.43677267e+03 5.27992515e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 0.07833557 -0.51920323 -0.00682128 -0.00380227  1.04236729]]


In [69]:

# Constant parameters used in Aruco methods
ARUCO_PARAMETERS = aruco.DetectorParameters_create()
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_50)

# Create grid board object we're using in our video
board = aruco.GridBoard_create(
        markersX=1,
        markersY=1,
        markerLength=64,
        markerSeparation=0.01,
        dictionary=ARUCO_DICT)

# Create vectors we'll be using for rotations and translations for postures
rotation_vectors, translation_vectors = None, None

#axis is just for visualisation of coordinate frame 
axis = np.float32([[-.5,-.5,0], [-.5,.5,0], [.5,.5,0], [.5,-.5,0],
                   [-.5,-.5,1],[-.5,.5,1],[.5,.5,1],[.5,-.5,1] ])

# https://stackoverflow.com/questions/46363618/aruco-markers-with-opencv-get-the-3d-corner-coordinates?rq=1
def rotate_marker_corners(rvec, markersize, tvec = None):

    mhalf = markersize / 2.0

    # convert rot vector to rot matrix both do: markerworld -> cam-world
    mrv, jacobian = cv2.Rodrigues(rvec)

    #in markerworld the corners are all in the xy-plane so z is zero at first
    X = mhalf * mrv[:,0] #rotate the x = mhalf
    Y = mhalf * mrv[:,1] #rotate the y = mhalf
    minusX = X * (-1)
    minusY = Y * (-1)

    # calculate 4 corners of the marker in camworld. corners are enumerated clockwise
    markercorners = []
    markercorners.append(np.add(minusX, Y)) #was upper left in markerworld
    markercorners.append(np.add(X, Y)) #was upper right in markerworld
    markercorners.append(np.add( X, minusY)) #was lower right in markerworld
    markercorners.append(np.add(minusX, minusY)) #was lower left in markerworld
    # if tvec given, move all by tvec
    if tvec is not None:
        C = tvec #center of marker in camworld
        for i, mc in enumerate(markercorners):
            markercorners[i] = np.add(C,mc) #add tvec to each corner
    #print('Vec X, Y, C, dot(X,Y)', X,Y,C, np.dot(X,Y)) # just for debug
    markercorners = np.array(markercorners,dtype=np.float32) # type needed when used as input to cv2
    return markercorners, mrv

In [ ]:
end_video_no =2
kinect_df = {}
for subject_number in range(1,end_video_no):
  kinect_df_ =pd.DataFrame(columns=['time_raw','kx', 'ky', 'kz', 'ax', 'ay', 'az' ])
  (height, width, layers) = (368,432,3)
  fps=14.97
  fourcc = cv2.VideoWriter_fourcc(*'MP42')
  video = cv2.VideoWriter("Subject "+str(subject_number)+"/Subject "+str(subject_number)+
                          "/Sub "+str(subject_number)+" SB/subject"+str(subject_number)+"_color_aruco.avi", fourcc, float(fps), (width, height))
  # video = cv2.VideoWriter("'drive'/'MyDrive'/'Aruco CMC project'/'subject1.mp4'", fourcc, float(fps), (width, height))
  i=0
  with open("Subject "+str(subject_number)+"/Subject "+str(subject_number)+"/Sub "+str(subject_number)+" SB/color.pickle",'rb') as color_img, \
       open("Subject "+str(subject_number)+"/Subject "+str(subject_number)+"/Sub "+str(subject_number)+" SB/xyz.pickle",'rb') as depth_img, \
       open("Subject "+str(subject_number)+"/Subject "+str(subject_number)+"/Sub "+str(subject_number)+" SB/prm.pickle",'rb') as prm:
      while True:
          try:
            # Change RGB to BGR
            color_image = pickle.load(color_img)
            Depth_xyz = pickle.load(depth_img)
            gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

            # Detect Aruco markers
            corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, ARUCO_DICT, parameters=ARUCO_PARAMETERS)
            # Refine detected markers
            # Eliminates markers not part of our board, adds missing markers to the board
            corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
                image = gray,
                board = board,
                detectedCorners = corners,
                detectedIds = ids,
                rejectedCorners = rejectedImgPoints,
                cameraMatrix = cameraMatrix,
                distCoeffs = distCoeffs)   
            ProjectImage = aruco.drawDetectedMarkers(color_image, corners, borderColor=(0, 0, 255))
            if ids is not None and len(ids) > 0:
              # Estimate the posture per each Aruco marker
              rotation_vectors, translation_vectors, _objPoints = aruco.estimatePoseSingleMarkers(corners, 1, cameraMatrix, distCoeffs)

              markerCorner = np.array((corners)).reshape((4, 2))
              (topLeft, topRight, bottomRight, bottomLeft) = markerCorner

              # convert each of the (x, y)-coordinate pairs to integers
              topRight = (int(topRight[0]), int(topRight[1]))
              bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
              bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
              topLeft = (int(topLeft[0]), int(topLeft[1]))

              # draw the bounding box of the ArUCo detection
              cv2.line(ProjectImage, topLeft, topRight, (0, 255, 0), 2)
              cv2.line(ProjectImage, topRight, bottomRight, (0, 255, 0), 2)
              cv2.line(ProjectImage, bottomRight, bottomLeft, (0, 255, 0), 2)
              cv2.line(ProjectImage, bottomLeft, topLeft, (0, 255, 0), 2)

              # compute and draw the center (x, y)-coordinates of the
              # ArUco marker
              cX = int((topLeft[0] + bottomRight[0]) / 2.0)
              cY = int((topLeft[1] + bottomRight[1]) / 2.0)
              cv2.circle(ProjectImage, (cX, cY), 4, (0, 0, 255), -1)

              # 3D Aruco Coordinates 
              # markercorners_3d, mrv = rotate_marker_corners(rotation_vectors, markersize = 64, tvec = translation_vectors)
              
              # markerCorner_xyz = np.array((markercorners_3d)).reshape((4, 3))
              # (topLeft, topRight, bottomRight, bottomLeft) = markerCorner_xyz

              # # convert each of the (x, y)-coordinate pairs to integers
              # topRight = (topRight[0], topRight[1], topRight[2])
              # bottomRight = (bottomRight[0], bottomRight[1], bottomRight[2])
              # bottomLeft = (bottomLeft[0], bottomLeft[1], bottomLeft[2])
              # topLeft = (topLeft[0], topLeft[1], topLeft[2])
              
              # # compute  the center (x, y, z)-coordinates of the
              # # ArUco marker
              # cX_3d = (topLeft[0] + bottomRight[0]) / 2.0
              # cY_3d = (topLeft[1] + bottomRight[1]) / 2.0
              # cZ_3d = (topLeft[2] + bottomRight[2]) / 2.0
              
              kinectxyz = np.append(pd.to_datetime(pickle.load(prm)), kinect2table(orig,R_T,np.array([Depth_xyz[cY][cX]])))
              # kxyz_axyz = np.append(kinectxyz, np.array([cX_3d, cY_3d, cZ_3d]))
              # print("Depth raw ", (Depth_xyz[cY][cX]))
              # print("tras vec ", translation_vectors)
              # print("k data ", kinect2table(orig,R_T,np.array([Depth_xyz[cY][cX]])))
 
              kxyz_axyz = np.append(kinectxyz, kinect2table(orig,R_T,translation_vectors))
              kinect_df_.loc[len(kinect_df_)] = kxyz_axyz

              # print("mark ", markercorners_)
              # print("\n  ", mrv)
            #   for rvec, tvec in zip(rotation_vectors, translation_vectors):
            #       if len(sys.argv) == 2 and sys.argv[1] == 'cube':
            #           try:
            #               imgpts, jac = cv2.projectPoints(axis, rvec, tvec, cameraMatrix, distCoeffs)
            #               # ProjectImage = drawCube(ProjectImage, corners, imgpts)
            #           except:
            #               continue
            #       else:    
            #         pass
            #           # ProjectImage = aruco.drawAxis(ProjectImage, cameraMatrix, distCoeffs, rvec, tvec, 1)
            # # cv2.imshow('ProjectImage', ProjectImage)

            # video.write(cv2.cvtColor(ProjectImage, cv2.COLOR_RGB2BGR))
            i = i+1
          except EOFError:
            kinect_df_['time'] = (kinect_df_['time_raw'] -kinect_df_['time_raw'][0]).dt.total_seconds()
            kinect_df[subject_number] = kinect_df_
            break
  # video.release()
  # !ffmpeg -y -loglevel info -i subject3_color.avi subject3_color.mp4

In [ ]:
kinect_df[1]

In [ ]:
#Get kinect and mocap data 
# Get the time delay between Kinect and Mocap 
# Trunkate the data
Time_delay={}
df_mocap={}
kinect_mocap = {}
for video_no in range(1,2):
  with open("Subject "+str(video_no)+"/Subject "+str(video_no)+"/Sub "+str(video_no)+" MC.csv") as Mocap_data:

      df_mocap[video_no] = pd.read_csv(Mocap_data)
      opti_time = datetime.strptime(df_mocap[video_no].columns[11], '%Y-%m-%d %I.%M.%S.%f %p')
 
      kinect_time = kinect_df[video_no]['time_raw'][0]

      print("Kinect Start", kinect_time," Opti Start",opti_time)
      Time_delay[video_no] = (kinect_time-opti_time).total_seconds()
      print(Time_delay[video_no])
      
      df_mocap[video_no].columns = df_mocap[video_no].iloc[4][:2].append(df_mocap[video_no].loc[1][2:].str.replace(r':', '_') + df_mocap[video_no].loc[4][2:])
      df_mocap[video_no] = df_mocap[video_no].iloc[5:].dropna(axis=1, how='all')
      df_mocap[video_no] = df_mocap[video_no].reset_index(drop=True)
      df_mocap[video_no] = df_mocap[video_no].astype(np.float64)
      df_mocap[video_no]['time_matched'] = round(df_mocap[video_no]['Time (Seconds)'] - round(Time_delay[video_no],2),2)
      
      kinect_mocap[video_no] = kinect_df[video_no].copy()
      kinect_mocap[video_no]['kx'] = kinect_mocap[video_no]['kx'].astype(int)
      kinect_mocap[video_no]['ky'] = kinect_mocap[video_no]['ky'].astype(int)
      kinect_mocap[video_no]['kz'] = kinect_mocap[video_no]['kz'].astype(int)
      kinect_mocap[video_no]['time_matched'] = round(kinect_mocap[video_no]['time'],2)
    
      kinect_mocap[video_no] = kinect_mocap[video_no].merge(df_mocap[video_no], how='left', on='time_matched')

In [ ]:
kinect_mocap[1]

In [ ]:
kinect_mocap[11] = kinect_mocap[1] - kinect_mocap[1].loc[0].values.squeeze()

# savgol filter
kinect_mocap[11] = kinect_mocap[11].apply(my_filter)
# Outlier Removal
kinect_mocap[11] = kinect_mocap[11][(np.abs(stats.zscore(kinect_mocap[11])) < 3).all(axis=1)]


# kinect_mocap[22] = kinect_mocap[2] - kinect_mocap[2].loc[0].values.squeeze()
# # savgol filter
# kinect_mocap[22] = kinect_mocap[22].apply(my_filter)
# # Outlier Removal
# kinect_mocap[22] = kinect_mocap[22][(np.abs(stats.zscore(kinect_mocap[22])) < 3).all(axis=1)]


In [ ]:
fig, ax = plt.subplots(3,3)
# Note that ax is now an array consisting of the individual axis
plt.xlabel('fps')
plt.ylabel('distance')

ax[0,0].plot(kinect_mocap[11]['ax'])
ax[0,0].set_title('ax')
ax[0,0].set_xlabel('fps')
ax[0,0].set_ylabel('distance')

ax[0,1].plot(kinect_mocap[11]['ay'])
ax[0,1].set_title('ay')
ax[0,1].set_xlabel('fps')
ax[0,1].set_ylabel('distance')

ax[0,2].plot(kinect_mocap[11]['az'])
ax[0,2].set_title('az')
ax[0,2].set_xlabel('fps')
ax[0,2].set_ylabel('distance')

ax[1,0].plot(kinect_mocap[11]['Skateboard_LX'])
ax[1,0].set_title('LX')
ax[1,0].set_xlabel('fps')
ax[1,0].set_ylabel('distance')

ax[1,1].plot(kinect_mocap[11]['Skateboard_LY'])
ax[1,1].set_title('LY')
ax[1,1].set_xlabel('fps')
ax[1,1].set_ylabel('distance')

ax[1,2].plot(kinect_mocap[11]['Skateboard_LZ'])
ax[1,2].set_title('LZ')
ax[1,2].set_xlabel('fps')
ax[1,2].set_ylabel('distance')

ax[2,0].plot(kinect_mocap[11]['kx'])
ax[2,0].set_title('KX')
ax[2,0].set_xlabel('fps')
ax[2,0].set_ylabel('distance')

ax[2,1].plot(kinect_mocap[11]['ky'])
ax[2,1].set_title('KY')
ax[2,1].set_xlabel('fps')
ax[2,1].set_ylabel('distance')

ax[2,2].plot(kinect_mocap[11]['kz'])
ax[2,2].set_title('KZ')
ax[2,2].set_xlabel('fps')
ax[2,2].set_ylabel('distance')



plt.tight_layout()